# Baseline 3
#### Training landmark and transcriptome models together

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import torch

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import StratifiedKFold

import itertools
import tqdm
import collections

## Prepare data:
#### Read joined data (pre + post treatment)

In [13]:
lincs_joined_df = pd.read_parquet("data/lincs_pretreatment.parquet")
kaggle_joined_df = pd.read_parquet("data/kaggle_pretreatment.parquet")
test_joined_df = pd.read_parquet("data/test_pretreatment.parquet")
print(f"lincs_joined_df = {lincs_joined_df.shape}\nkaggle_joined_df = {kaggle_joined_df.shape}\ntest_joined_df = {test_joined_df.shape}")

lincs_joined_df = (107404, 1842)
kaggle_joined_df = (602, 1841)
test_joined_df = (255, 921)


In [14]:
all_train_df = pd.concat([kaggle_joined_df,lincs_joined_df])
print(f"all_train_df = {all_train_df.shape}")

all_train_df = (108006, 1843)


#### Kaggle provided data

In [15]:
de_train = pd.read_parquet('data/de_train.parquet')
id_map = pd.read_csv('data/id_map.csv',index_col='id')

#### Define features of interest and sort data accordingly.

In [16]:
features = ['cell_type', 'sm_name']
multiindex_features = [("label",'cell_type'),("label",'sm_name')]

transcriptome_cols = de_train.columns[5:]
landmark_cols = kaggle_joined_df["post_treatment"].columns
print(f"transcriptome_cols = {transcriptome_cols.shape}\nlandmark_cols = {landmark_cols.shape}")

transcriptome_cols = (18211,)
landmark_cols = (918,)


In [17]:
# We only need to sort these two dataframes because they represent the same underlying dataset.
de_train = de_train.query("~control").sort_values(features)
kaggle_joined_df = kaggle_joined_df.sort_values(multiindex_features)
# Sanity check that these dfs align.
genes_align = (kaggle_joined_df["post_treatment"] == de_train[landmark_cols]).all(axis=None)
labels_align = (kaggle_joined_df["label"][features] == de_train[features]).all(axis=None)
genes_align and labels_align

True

## Prepare for training
#### Partition data

In [18]:
eval_cells_only_df = kaggle_joined_df[kaggle_joined_df["label"]["cell_type"].isin(["B cells", "Myeloid cells"])].reset_index(drop=True)
mean_score = pd.concat([eval_cells_only_df["label"]["sm_name"],eval_cells_only_df["post_treatment"]],axis=1).groupby("sm_name").agg("mean").mean(axis=1)
classes = np.digitize(mean_score.values, bins=[0,0.1, 0.5, 1])
cpds = mean_score.index.values

# Don't need this df it just looks cool
mean_score.to_frame(name="mean_score").assign(cv_class=classes)

,mean_score,cv_class
sm_name,,
Alvocidib,1.144652,4
CHIR-99021,0.145969,2
Crizotinib,-0.010615,0
Dactolisib,-0.077123,0
Foretinib,-0.007336,0
Idelalisib,0.013947,1
LDN 193189,0.070423,1
Linagliptin,0.033001,1
MLN 2238,2.045447,4


#### CV splits

In [19]:
fold_arr = np.full(len(cpds), -1)
n_splits = 3
skf = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)
for fold, (_, val_ind) in enumerate(skf.split(classes,classes)):
    fold_arr[val_ind] = fold
fold_map = {c: f for c, f in zip(cpds, fold_arr)}
fold_to_cpds = {fold: cpds[fold_arr==fold] for fold in range(n_splits)}

fold_to_cpds

/Users/laurasisson/miniconda3/envs/rdenv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


{0: array(['Alvocidib', 'Crizotinib', 'Foretinib', 'LDN 193189', 'R428'],
       dtype=object),
 1: array(['CHIR-99021', 'MLN 2238', 'Palbociclib', 'Penfluridol',
        'Porcn Inhibitor III'], dtype=object),
 2: array(['Dactolisib', 'Idelalisib', 'Linagliptin',
        'O-Demethylated Adapalene', 'Oprozomib (ONX 0912)'], dtype=object)}

In [20]:
identity_transformer = FunctionTransformer()
X = np.array([[0, 1], [2, 3]])
X
identity_transformer.inverse_transform(X), identity_transformer.transform(X)

(array([[0, 1],
        [2, 3]]),
 array([[0, 1],
        [2, 3]]))

#### Model helper functions

In [21]:
def make_transformer(n_components):
    if n_components == 0:
        # Identity transformer
        return FunctionTransformer()
    else:
        return TruncatedSVD(n_components=n_components, random_state=1)

# Unfortunately, sklearn doesn't allow us to use a Ridge model as an intermediate step
# because intermediate steps are supposed to be preprocessors
def make_landmark_model(params):
    params = params["lmk"]
    svd = make_transformer(params["n_comp"])
    model = make_pipeline(ColumnTransformer([('ohe', OneHotEncoder(), list(params["encoded"]))]),
                          Ridge(alpha=params["alpha"], fit_intercept=False))
    return svd, model

def make_transcriptome_model(params):
    params = params["trm"]
    svd = make_transformer(params["n_comp"])
    model = Ridge(alpha=params["alpha"], fit_intercept=False)
    return svd, model

# This changes the models inplace. Returns as a courtesy.
def model_train(mp,x,y):
    svd, model = mp
    z = svd.fit_transform(y)
    model.fit(x, z)
    return svd, model

def model_predict(mp,x):
    svd, model = mp
    d = model.predict(x)
    return svd.inverse_transform(d)
    

#### Draw some example models

In [22]:
example_params = {"lmk":{"n_comp":150,"alpha":.1,"encoded":("sm_name",)},"trm":{"n_comp":150,"alpha":.1}}
print("Landmark")
display(make_landmark_model(example_params))
print("Transcriptome")
display(make_transcriptome_model(example_params))

Landmark


(TruncatedSVD(n_components=150, random_state=1),
 Pipeline(steps=[('columntransformer',
                  ColumnTransformer(transformers=[('ohe', OneHotEncoder(),
                                                   ['sm_name'])])),
                 ('ridge', Ridge(alpha=0.1, fit_intercept=False))]))

Transcriptome


(TruncatedSVD(n_components=150, random_state=1),
 Ridge(alpha=0.1, fit_intercept=False))

#### Cross Validation function

In [23]:
def crossvalidate(params, print_each=False, print_result=False):
    def train_landmark(cpds):
        mask_va = all_train_df[("label",'cell_type')].isin(['Myeloid cells', 'B cells']) & all_train_df[("label",'sm_name')].isin(cpds)
        validate = all_train_df[mask_va]
    
        mask_tr = ~mask_va
        train = all_train_df[mask_tr]

        svd, model = model_train(make_landmark_model(params), train["label"][features], train["post_treatment"])
        return svd, model

    def train_transcriptome(landmark_mp,cpds):
        mask_va = kaggle_joined_df[("label",'cell_type')].isin(['Myeloid cells', 'B cells']) & kaggle_joined_df[("label",'sm_name')].isin(cpds)
        mask_tr = ~mask_va # 485 or 487 training rows

        train_x = model_predict(landmark_mp, kaggle_joined_df[mask_tr]["label"][features])
        train_y = de_train[mask_tr][transcriptome_cols]

        val_x = model_predict(landmark_mp, kaggle_joined_df[mask_va]["label"][features])
        val_y = de_train[mask_va][transcriptome_cols]

        transcriptome_mp = model_train(make_transcriptome_model(params), train_x, train_y)
        pred_y = model_predict(transcriptome_mp, val_x)
        mrrmse = np.sqrt(np.square(val_y - pred_y).mean(axis=1)).mean()
        return mrrmse
    
    mrrmse_list = []
    for fold, cpds in fold_to_cpds.items():
        landmark_mp = train_landmark(cpds)
        mrrmse = train_transcriptome(landmark_mp,cpds)
        mrrmse_list.append(mrrmse)
        if print_each:
            print(f"# Fold {fold}: {mrrmse:5.3f}")

    mrrmse = np.array(mrrmse_list).mean()
    if print_result:
        print(f"# Overall {mrrmse:5.3f} {params}")
    return mrrmse
                
crossvalidate(example_params, print_each=True)

# Fold 0: 2.012
# Fold 1: 2.666
# Fold 2: 2.172


2.283452586892314

In [24]:
import itertools

alpha_list = [0.1, 1, 5] #10
encoded = [('sm_name',)] #,('cell_type',),('sm_name','cell_type')
n_comps_list = [0,10,30,100]


all_trials = [{"lmk":{"n_comp":lmk_n_comp,"alpha":lmk_alpha,"encoded":encoded},"trm":{"n_comp":trm_n_comp,"alpha":trm_alpha}} for encoded, lmk_alpha, lmk_n_comp, trm_alpha, trm_n_comp in itertools.product(encoded,alpha_list,n_comps_list,alpha_list,n_comps_list)]
print(f"Generated {len(all_trials)} trials.\nExample = {next(iter(all_trials))}")

Generated 144 trials.
Example = {'lmk': {'n_comp': 0, 'alpha': 0.1, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 0, 'alpha': 0.1}}


In [28]:
best_score = float('inf')
best_params = dict()
scores = collections.Counter()
for i,params in enumerate(tqdm.tqdm(all_trials,smoothing=0)):
    score = crossvalidate(params)
    if score < best_score:
        best_score = score
        best_params = params
    scores[i] = -score
for i, score in scores.most_common(5):
    print(all_trials[i],score)
print(f"Best score = {best_score}.\nBest params = {best_params}")

100%|███████████████████████████████████████| 144/144 [1:16:24<00:00, 31.84s/it]

{'lmk': {'n_comp': 10, 'alpha': 5, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 30, 'alpha': 0.1}} -2.259654876047867
{'lmk': {'n_comp': 10, 'alpha': 5, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 30, 'alpha': 1}} -2.25968089986029
{'lmk': {'n_comp': 10, 'alpha': 5, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 30, 'alpha': 5}} -2.2598107983252733
{'lmk': {'n_comp': 10, 'alpha': 5, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 100, 'alpha': 0.1}} -2.260035200635452
{'lmk': {'n_comp': 10, 'alpha': 5, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 100, 'alpha': 1}} -2.260060465738705
Best score = 2.259654876047867.
Best params = {'lmk': {'n_comp': 10, 'alpha': 5, 'encoded': ('sm_name',)}, 'trm': {'n_comp': 30, 'alpha': 0.1}}


## Submission
We retrain the model on the full training data and create a submission file.

In [29]:
lm_mp = model_train(make_landmark_model(best_params), all_train_df["label"][features], all_train_df["post_treatment"])

train_x = model_predict(lm_mp, kaggle_joined_df["label"][features])
train_y = de_train[transcriptome_cols]

trm_mp = model_train(make_transcriptome_model(best_params), train_x, train_y)

Make submission

In [30]:
# Quick sanity check to make sure our submission is ordered correctly.
assert (test_joined_df["label"][features] == id_map[features]).all(axis=None)

# Full pipeline. Nice
test_lm_pred = model_predict(lm_mp, test_joined_df["label"][features])
y_pred = model_predict(trm_mp, test_lm_pred)

y_pred.shape
submission = pd.DataFrame(y_pred, columns=transcriptome_cols, index=id_map.index)
display(submission)
submission.to_csv('submissions/pipeline.csv')

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
id,,,,,,,,,,,,,,,,,,,,,
0,0.390133,0.223634,0.350399,0.553414,1.082215,1.026326,-0.026304,0.245155,-0.278573,0.363279,...,-0.258527,0.018024,0.221187,0.151393,0.465893,0.296939,0.262734,0.161673,-0.221435,0.073930
1,0.063709,0.042970,0.045235,0.056776,0.145158,0.241273,-0.003462,0.029868,-0.007784,-0.036490,...,0.001094,0.011096,0.039175,0.059650,0.076324,0.051956,0.017792,-0.007389,-0.009626,-0.010535
2,0.234387,0.155509,0.183633,0.232050,0.558107,0.864019,-0.011177,0.116036,-0.070272,-0.014906,...,-0.080871,0.024740,0.131348,0.172717,0.258872,0.178563,0.081725,0.022808,-0.071225,-0.005349
3,0.023762,0.012014,0.017928,0.041771,0.072239,0.057522,-0.005611,0.015934,-0.027334,0.040132,...,-0.016514,-0.001387,0.018943,0.008984,0.034147,0.020776,0.022054,0.014102,-0.014536,-0.001393
4,0.067793,0.026514,0.057324,0.075272,0.075080,0.181430,-0.010098,0.098876,-0.047431,0.181914,...,-0.080207,0.017562,0.015412,0.000892,0.075538,0.080602,0.088491,0.123945,-0.064057,0.009298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,-0.210265,-0.121665,-0.102966,-0.083616,-0.435236,-0.714579,-0.035168,-0.106599,-0.028784,0.068409,...,-0.010179,-0.077949,-0.098151,-0.184331,-0.206026,-0.142519,-0.042762,0.006355,0.014855,-0.009723
251,0.089308,0.062887,0.104530,0.167607,0.327301,0.235901,-0.003540,0.038307,-0.096987,0.140004,...,-0.112091,-0.018052,0.062527,0.014616,0.093823,0.051047,0.050799,0.035386,-0.056609,0.036316
252,-0.077926,-0.037589,-0.025215,-0.033343,-0.121284,-0.330512,-0.012004,-0.029334,0.005664,-0.000199,...,0.033136,-0.022008,-0.022308,-0.055505,-0.048807,-0.037224,-0.008905,-0.015518,0.000818,0.002451
